In [1]:
import pandas as pd
import numpy as np

data_info = pd.read_csv('data/data_info.csv')
train = pd.read_parquet('data/train.parquet')
test = pd.read_parquet('data/test.parquet')
submission = pd.read_csv('data/sample_submission.csv')

print(data_info.shape, train.shape, test.shape, submission.shape)

(24, 2) (4701217, 25) (291241, 24) (291241, 2)


In [2]:
def data_processing(exdata):
  del exdata['id']
  del exdata['vehicle_restricted']
  del exdata['height_restricted']
  del exdata['start_latitude']
  del exdata['end_latitude']
  del exdata['start_longitude']
  del exdata['end_longitude']


  return exdata

train = data_processing(train)
test = data_processing(test)

In [3]:
train = train[train.maximum_speed_limit != 40.0]

In [4]:
mask = train['road_name'].isin(['지방도1112호선', '중산간서로','지방도1119호선','지방도1120호선','삼무로','지방도1116호선', '일반국도95호선','남조로',
'신광로','임항로', '애원로','일주동로', '호서중앙로','아봉로', '호근로' ])
train = train[~mask]

In [5]:
train = train.replace({'road_name':{'연삼로':1,'일반국도12호선':2,'일반국도16호선':3,'일반국도11호선':4,'번영로':5,'수영장길':6,'-':7,
'지방도1136호선':8,'제2거로교':9,'일반국도99호선':10,'경찰로':11,'시청로':12,'서사로':13,'관광단지1로':14,'신산로':15,'새서귀로':16,
'중문로':17,'지방도1132호선':18,'태평로':19,'산서로':20,'관광단지로':21,'중정로':22,'관광단지2로':23,'동홍로':24,'고평교':25,'애조로':26,'삼봉로':27,
'외도천교':28,'권학로':29,'신대로':30,'삼성로':31,'지방도1118호선':32,'시민광장로':33,'지방도1115호선':34,'중앙로':35,'첨단로':36,'관덕로':37,'어시천교':38,
'한천로':39,'동부관광도로':40,'연북2교':41,'동문로':42,'연북로':43,'연동로':44,'지방도1117호선':45,'지방도97호선':46}})

test = test.replace({'road_name':{'연삼로':1,'일반국도12호선':2,'일반국도16호선':3,'일반국도11호선':4,'번영로':5,'수영장길':6,'-':7,
'지방도1136호선':8,'제2거로교':9,'일반국도99호선':10,'경찰로':11,'시청로':12,'서사로':13,'관광단지1로':14,'신산로':15,'새서귀로':16,
'중문로':17,'지방도1132호선':18,'태평로':19,'산서로':20,'관광단지로':21,'중정로':22,'관광단지2로':23,'동홍로':24,'고평교':25,'애조로':26,'삼봉로':27,
'외도천교':28,'권학로':29,'신대로':30,'삼성로':31,'지방도1118호선':32,'시민광장로':33,'지방도1115호선':34,'중앙로':35,'첨단로':36,'관덕로':37,'어시천교':38,
'한천로':39,'동부관광도로':40,'연북2교':41,'동문로':42,'연북로':43,'연동로':44,'지방도1117호선':45,'지방도97호선':46}})

In [6]:
from sklearn.preprocessing import LabelEncoder

str_col = ['base_date', 'day_of_week', 'base_hour', 'road_in_use', 'lane_count',
       'road_rating', 'road_name', 'multi_linked', 'connect_code',
       'maximum_speed_limit', 'weight_restricted', 'road_type',
       'start_node_name', 'start_turn_restricted', 'end_node_name',
       'end_turn_restricted']
       
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [7]:
from sklearn.preprocessing import MinMaxScaler
minmaxScaler = MinMaxScaler()
train['distance_scaled'] = minmaxScaler.fit_transform(train['distance'].values.reshape(-1,1))
test['distance_scaled'] = minmaxScaler.fit_transform(test['distance'].values.reshape(-1,1))

del train['distance']
del test['distance']

In [8]:
X_train = train.drop("target",axis=1)
y_train = train['target']
y_test = submission['target']

In [9]:
X_train.shape, test.shape, y_train.shape, y_test.shape

((4006268, 17), (291241, 17), (4006268,), (291241,))

In [10]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import random


# 2. 모델 구성하기
model = Sequential()
model.add(Dense(256, input_shape=X_train.shape[1:], activation='swish'))
model.add(Dense(128, activation='swish'))
model.add(Dense(64, activation='swish'))
model.add(Dense(32, activation='swish'))
model.add(Dense(1))

# 3. 모델 학습과정 설정하기
model.compile(optimizer='adam', loss='mae')

# 4. 모델 학습시키기
hist = model.fit(X_train, y_train, epochs=30, batch_size=128)
y_pred = model.predict(test)

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-15 05:00:25.144551: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-15 05:00:25.144914: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30


2022-12-15 05:00:27.310521: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-15 05:00:27.834373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


31299/31299 [==============================] - 411s 13ms/step - loss: 6.1129
Epoch 2/30
31299/31299 [==============================] - 397s 13ms/step - loss: 4.6863
Epoch 3/30
31299/31299 [==============================] - 407s 13ms/step - loss: 4.5137
Epoch 4/30
31299/31299 [==============================] - 375s 12ms/step - loss: 4.4265
Epoch 5/30
31299/31299 [==============================] - 379s 12ms/step - loss: 4.3626
Epoch 6/30
31299/31299 [==============================] - 372s 12ms/step - loss: 4.3122
Epoch 7/30
31299/31299 [==============================] - 370s 12ms/step - loss: 4.2747
Epoch 8/30
31299/31299 [==============================] - 374s 12ms/step - loss: 4.2444
Epoch 9/30
31299/31299 [==============================] - 373s 12ms/step - loss: 4.2151
Epoch 10/30
31299/31299 [==============================] - 379s 12ms/step - loss: 4.1894
Epoch 11/30
31299/31299 [==============================] - 295s 9ms/step - loss: 4.1688
Epoch 12/30
31299/31299 [=================

2022-12-15 07:26:10.854006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9102/9102 [==============================] - 20s 2ms/step


In [11]:
submission['target'] = y_pred

In [12]:
submission

,id,target
0,TEST_000000,27.994806
1,TEST_000001,43.184551
2,TEST_000002,52.797756
3,TEST_000003,36.886700
4,TEST_000004,43.575874
...,...,...
291236,TEST_291236,49.459301
291237,TEST_291237,50.140625
291238,TEST_291238,22.800591
291239,TEST_291239,22.373558


In [13]:
submission.to_csv('deep_final_v6.csv',index=False)